In [ ]:
import requests
import json
import pyttsx3
import speech_recognition as sr
from datetime import datetime, timedelta
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

# Escopos necessários para a API do Google Calendar
SCOPES = ['https://www.googleapis.com/auth/calendar']

def get_current_datetime():
    """Obtém a data e hora atuais."""
    now = datetime.now()
    return now.strftime("%d/%m/%Y %H:%M")

# Obtém a data/hora atual ao iniciar
current_time = get_current_datetime()

def query_mistral(prompt):
    """Consulta o modelo Mistral via API local do Ollama."""
    try:
        # Configuração do payload para a requisição
        data = {
            "model": "mistral",
            "messages": [{"role": "user", "content": prompt}],
            "stream": False
        }
        # Adiciona timeout para evitar requisições penduradas
        response = requests.post("http://localhost:11434/api/chat", json=data, timeout=10)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Erro na consulta ao Mistral: {e}")
        return None

def speak_text(text):
    """Converte texto em voz usando pyttsx3."""
    engine = pyttsx3.init()
    # Configuração da voz
    engine.setProperty('rate', 150)    # Velocidade da fala
    engine.setProperty('volume', 0.9)  # Volume (0.0 a 1.0)
    engine.say(text)
    engine.runAndWait()

def get_voice_input():
    """Captura entrada de voz com comando explícito para parar."""
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("Listening... Say 'stop recording' to end.")
        full_text = ""

        while True:
            try:
                # Configuração do microfone e timeout
                audio = recognizer.listen(source, timeout=None)
                text = recognizer.recognize_google(audio).lower()
                print(f"Heard: {text}")

                # Verifica comando para parar gravação
                if "stop recording" in text:
                    print("Stopping recording.")
                    break

                full_text += text + ". "

            except sr.UnknownValueError:
                print("Não foi possível entender o áudio.")
            except sr.RequestError as e:
                print(f"Erro no serviço de reconhecimento: {e}")
                return None

        return full_text.strip()

def authenticate_google():
    """Autentica com a API do Google usando OAuth2."""
    creds = None
    # Fluxo de autenticação local
    flow = InstalledAppFlow.from_client_secrets_file('credentials_calendar_automation.json', SCOPES)
    creds = flow.run_local_server(port=0)
    return build('calendar', 'v3', credentials=creds)

def create_event(service, title, date, time):
    """Cria um evento no Google Calendar."""
    # Usa data atual se não for especificada
    if not date or date.strip() == "":
        date = get_current_datetime().split(" ")[0]

    # Conversão para objeto datetime
    event_start = datetime.strptime(f"{date} {time}", "%d/%m/%Y %H:%M")
    event_end = event_start + timedelta(hours=1)  # Evento de 1 hora

    # Estrutura do evento
    event = {
        'summary': title,
        'start': {
            'dateTime': event_start.isoformat(),
            'timeZone': 'America/Sao_Paulo',
        },
        'end': {
            'dateTime': event_end.isoformat(),
            'timeZone': 'America/Sao_Paulo',
        },
        'reminders': {
            'useDefault': False,
            'overrides': [
                {'method': 'email', 'minutes': 30},  # Lembrete por email
                {'method': 'popup', 'minutes': 10},  # Lembrete por popup
            ],
        },
    }

    # Insere o evento no calendário
    event_result = service.events().insert(calendarId='primary', body=event).execute()
    print(f"Event created: {event_result.get('htmlLink')}")
    return event_result.get('htmlLink')

def main():
    # Autenticação inicial
    service = authenticate_google()

    while True:
        print("Speak your request (or type 'exit' to quit):")
        user_input = get_voice_input()
        if user_input is None:
            continue

        # Comando para sair do programa
        if "exit" in user_input.lower():
            print("Exiting program.")
            break

        # Prompt estruturado para o Mistral
        prompt = f"Considering today's date and time ({current_time}) Extract action ,time(hh:mm), date(dd/mm/yyyy), and description from: {user_input}, output only the following format Appointment title: title / Date: dd/mm/yyyy / Time: hh:mm"
        parsed_json = query_mistral(prompt)

        if parsed_json:
            try:
                # Processa a resposta do LLM
                ai_reply = parsed_json.get("message", {}).get("content", "")
                print("\nAI Response:")
                print(ai_reply)

                if ai_reply:
                    # Extrai informações da resposta
                    lines = ai_reply.split("/ ")
                    appointment_title = lines[0].replace("Appointment title: ", "").strip()
                    appointment_date = lines[1].replace("Date: ", "").strip()
                    appointment_time = lines[2].replace("Time: ", "").strip()

                    # Limpeza de possíveis artefatos na resposta
                    if "(" in appointment_time:
                        appointment_time = appointment_time.split("(")[0].strip()

                    # Fallback para data atual
                    if not appointment_date:
                        appointment_date = get_current_datetime().split(" ")[0]

                    # Confirmação dos dados
                    print("\nParsed Variables:")
                    print(f"Title: {appointment_title}")
                    print(f"Date: {appointment_date if appointment_date else 'Today'}")
                    print(f"Time: {appointment_time}")

                    # Criação do evento
                    event_link = create_event(service, appointment_title, appointment_date, appointment_time)

                    # Feedback por voz
                    success_message = f"Your appointment titled '{appointment_title}' has been successfully created."
                    print(success_message)
                    speak_text(success_message)

                else:
                    print("No valid response from AI.")

            except KeyError as e:
                print(f"Error parsing response: {e}")
                print("Unexpected response format. Full response:")
                print(parsed_json)
        else:
            print("Failed to parse input. Please try again.")

if __name__ == '__main__':
    main()